# Group Project / Assignment 4: Instruction finetuning a Llama-3.2 model
**Assignment due 21 April 11:59pm**

Welcome to the fourth and final assignment for 50.055 Machine Learning Operations. The third and fourth assignment together form the course group project. You will continue the work on a chatbot which can answer questions about SUTD to prospective students.


**This assignment is a group assignment.**

- Read the instructions in this notebook carefully
- Add your solution code and answers in the appropriate places. The questions are marked as **QUESTION:**, the places where you need to add your code and text answers are marked as **ADD YOUR SOLUTION HERE**. The assignment is more open-ended than previous assignments, i.e. you have more freedom how to solve the problem and how to structure your code.
- The completed notebook, including your added code and generated output will be your submission for the assignment.
- The notebook should execute without errors from start to finish when you select "Restart Kernel and Run All Cells..". Please test this before submission.
- Use the SUTD Education Cluster to solve and test the assignment. If you work on another environment, minimally test your work on the SUTD Education Cluster.

**Rubric for assessment** 

Your submission will be graded using the following criteria. 
1. Code executes: your code should execute without errors. The SUTD Education cluster should be used to ensure the same execution environment.
2. Correctness: the code should produce the correct result or the text answer should state the factual correct answer.
3. Style: your code should be written in a way that is clean and efficient. Your text answers should be relevant, concise and easy to understand.
4. Partial marks will be awarded for partially correct solutions.
5. Creativity and innovation: in this assignment you have more freedom to design your solution, compared to the first assignments. You can show of your creativity and innovative mindset. 
6. There is a maximum of 310 points for this assignment.

**ChatGPT policy** 

If you use AI tools, such as ChatGPT, to solve the assignment questions, you need to be transparent about its use and mark AI-generated content as such. In particular, you should include the following in addition to your final answer:
- A copy or screenshot of the prompt you used
- The name of the AI model
- The AI generated output
- An explanation why the answer is correct or what you had to change to arrive at the correct answer

**Assignment Notes:** Please make sure to save the notebook as you go along. Submission Instructions are located at the bottom of the notebook.



### Finetuning LLMs

The goal of the assignment is to build a more advanced chatbot that can talk to prospective students and answer questions about SUTD.

We will finetune a smaller 1B LLM on question-answer pairs which we synthetically generate. Then we will compare the finetuned and non-finetuned LLMs with and without RAG to see if we were able to improve the SUTD chatbot answer quality. 

We'll be leveraging `langchain`, `llama 3.2` and `Google AI STudio with Gemini 2.0`.

Check out the docs:
- [LangChain](https://docs.langchain.com/docs/)
- [Llama 3.2](https://www.llama.com/docs/model-cards-and-prompt-formats/llama3_2/)
- [Google AI Studio](https://aistudio.google.com/)

Note: Google AI Studio provides a lot of free tokens but has certain rate limits. Write your code in a way that it can handle these limits.

# Install dependencies
Use pip to install all required dependencies of this assignment in the cell below. Make sure to test this on the SUTD cluster as different environments have different software pre-installed.  

In [16]:
# QUESTION: Install and import all required packages
# The rest of your code should execute without any import or dependency errors.

# **--- ADD YOUR SOLUTION HERE (10 points) ---**
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers
!pip install datasets
!pip install accelerate
!pip install huggingface_hub
!pip install langchain langchain-core langchain-community langchain-experimental langchain-google-genai
!pip install google-generativeai
!pip install flashrank
!pip install openai
!pip install sentence-transformers
!pip install dotenv
!pip install pydantic

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached google_ai_generativelanguage-0.6.17-py3-none-any.whl.metadata (9.8 kB)
Using cached google_ai_generativelanguage-0.6.17-py3-none-any.whl (1.4 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.17 which is incompatible.


  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.17
    Uninstalling google-ai-generativelanguage-0.6.17:
      Successfully uninstalled google-ai-generativelanguage-0.6.17


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.2 requires google-ai-generativelanguage<0.7.0,>=0.6.16, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


# Importing libraries

In [12]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
# TODO: check the difference between ChatGoogleGenerativeAI and GoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.chains import LLMChain
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
import os
from pydantic import BaseModel
import time
import json

load_dotenv()
GOOGLE_GENAI_API_KEY = os.getenv("GOOGLE_GENAI_API_KEY")

# Generate training data
The first step of the assignment is generating synthetic question-answer pairs which can be used for finetuning an LLM model. 
Use the Google AI studio with the Gemini models to create -high-quality QA training data.


In [13]:
# QUESTION: Use langchain and the Google AI Studio APIs and a model from the Gemini 2.0 family
# to create a text-generation chain that can produce and parse JSON output.
# Test it by having the LLM generate a JSON array of 3 fruits

#--- ADD YOUR SOLUTION HERE (20 points)---

# --------------------------------------------------------------------------------

# we are creating synthetic data that contains only questions and responses but not on multiple tasks, because this is a chatbot application
model = ChatGoogleGenerativeAI(google_api_key=GOOGLE_GENAI_API_KEY, model="gemini-2.0-flash", temperature=0.2, convert_system_message_to_human=True)
parser = JsonOutputParser()
input_prompt = "Generate a JSON array containing exactly 3 fruit names."
prompt = ChatPromptTemplate.from_messages([
    ("system", """
    You are a helpful assistant that generates structured JSON data.
    Your response should ONLY contain valid JSON without any additional text.
    Do not include explanation, notes, or markdown formatting.
    """),
    ("human", "{input_text}")
])
chain = ( prompt | model | parser )
response = chain.invoke({"input_text": input_prompt})
print(response)

# --------------------------------------------------------------------------------

# imo there is a better way to do this which is using pydantic and langchain
class FruitList(BaseModel):
    fruits: List[str] = Field(description="A list of fruit objects", min_length=3, max_length=3)
    
pydantic_parser = JsonOutputParser(pydantic_object=FruitList)
prompt = PromptTemplate(
    template  = "Answer the user's question.\n\n{format_instructions}\n\n{input}",
    input_variables = ["input"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
   
chain = ( prompt | model | pydantic_parser )
response = chain.invoke({"input": input_prompt})
print(response)

c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


['apple', 'banana', 'orange']


c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


['apple', 'banana', 'orange']


## Generate topics
When generating data, it is often helpful to guide the generation process through some hierachical structure. 
Before we create question-answer pairs, let's generate some topics which the questions should be about.



In [32]:
# QUESTION: Create a function 'generate_topics' which generates topics which prospective students might care about.
#
# Generate a list of 20 topics 

#--- ADD YOUR SOLUTION HERE (20 points)---
def generate_topics(num = 20):
    class TopicsList(BaseModel):
        topics: List[str] = Field(description=f"A list of {num} topics that prospective students might care about", min_length=num, max_length=num)
    
    # if the file already exists don't run instead return the topics from the file
    output_dir = "data"
    
    if os.path.exists(output_dir) == False:
        os.makedirs(output_dir)
        
    filename = f"data/topics.json"
            
    model = ChatGoogleGenerativeAI(
        google_api_key=GOOGLE_GENAI_API_KEY,
        model="gemini-2.0-flash",
        temperature=0.8, # using this value for more diverse topics
        convert_system_message_to_human=True
    )    
    
    pydantic_parser = JsonOutputParser(pydantic_object=TopicsList)
    prompt = PromptTemplate(
        template="Generate a list of topics that prospective students might care about when choosing a college or university.\n\n{format_instructions}\n\n{input}",
        input_variables=["input"],
        partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
    )
    input_prompt = f"Please generate exactly {num} diverse topics that prospective students would consider when choosing a college. Include academic, social, financial, and career-related aspects."
    
    chain = prompt | model | parser 
    response = chain.invoke({"input": input_prompt})

    topics = response["topics"]
    
    # save to disk otherwise
    with open(filename, 'w') as file:
        json.dump(topics, file)
    
    return topics

In [33]:
# test topic generation
print(generate_topics(3))

c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='A list of 3 topics that prospective students might care about' min_length=3 max_length=3 extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


['Academic Program Quality and Reputation', 'Campus Culture and Social Opportunities', 'Cost of Attendance and Financial Aid']


In [34]:
# Generate a list of 20 topics 
# We save a copy to disk and reload it from there if the file exists

# the function to generate topics and saves it to disk as topics_20.json
print(generate_topics(20))

c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='A list of 20 topics that prospective students might care about' min_length=20 max_length=20 extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


['Academic Reputation', 'Specific Programs/Majors Offered', 'Faculty Expertise and Accessibility', 'Research Opportunities', 'Internship Opportunities', 'Career Services and Placement Rates', 'Tuition and Fees', 'Financial Aid and Scholarships', 'Cost of Living', 'Campus Location and Safety', 'Campus Culture and Social Life', 'Student Organizations and Clubs', 'Diversity and Inclusion', 'Student-Faculty Ratio', 'Class Sizes', 'Graduation Rates', 'Alumni Network', 'Housing Options', 'Athletic Programs', 'Study Abroad Programs']


## Generate questions
Now generate a set of questions about each topic

In [ ]:
# QUESTION: Create a function 'generate_questions' which generates quetions about a given topic. 
# Generate a list of 10 questions per topics. In total you should have 200 questions. 
#

#--- ADD YOUR SOLUTION HERE (20 points)---
# TODO: rememeber to add timeout to the function
def generate_questions(topic, num=10):
    output_dir = "data"
    
    if os.path.exists(output_dir) == False:
        os.makedirs(output_dir)

    model = ChatGoogleGenerativeAI(
        google_api_key=GOOGLE_GENAI_API_KEY,
        model="gemini-2.0-flash",
        temperature=0.4, # hyperparameter can be changed
        convert_system_message_to_human=True
    )    
    
    class Questions(BaseModel):
        questions: List[str] = Field(
            description=f"A list of {num} questions about a specific topic", 
            min_length=num, 
            max_length=num
        )
    
    parser = JsonOutputParser(pydantic_object=Questions)
    prompt = PromptTemplate(
        template="Generate a list of thought-provoking questions about the following topic that prospective students might ask.\n\nTopic: {topic}\n\n{format_instructions}\n\nMake sure to generate exactly {num_questions} diverse, specific, and insightful questions.",
        input_variables=["topic", "num_questions"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    chain = prompt | model | parser   
    
    questions_data = {}
    
    try:
        print(f"Generating questions for topic: {topic}")
        response = chain.invoke({"topic": topic, "num_questions": num})
        questions_data[topic] = response["questions"]
        
    except Exception as e:
        print(f"Error generating questions for topic '{topic}': {e}")

    return questions_data

In [25]:
# test it
print(generate_questions("Academic Reputation and Program Quality", 3))

c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='A list of 3 questions about a specific topic' min_length=3 max_length=3 extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Generating questions for topic 39: Academic Reputation and Program Quality
{'Academic Reputation and Program Quality': ['Beyond rankings, how does the program foster a culture of intellectual curiosity and rigorous scholarship among its students and faculty?', 'What specific initiatives are in place to ensure the curriculum remains current and relevant to industry trends and emerging research areas within the field?', 'How does the program support students in developing critical thinking and problem-solving skills applicable to real-world challenges, and can you provide examples of successful alumni who have leveraged these skills?']}


In [26]:
# # QUESTION: Now let's put it together and generate 10 questions for each topic. Save the questions in a local file.

#--- ADD YOUR SOLUTION HERE (20 points)---
with open("data/topics_20.json", "r") as f:
    topics = json.load(f)

rate_limit = 15
requests_made = 0
all_questions = {}

for i, topic in enumerate(topics):
    if requests_made >= rate_limit:
        time.sleep(60)
        requests_made = 0
    response = generate_questions(topic)
    all_questions[topic] = response[topic]
    requests_made += 1

with open("data/questions.json", "w") as f:
    json.dump(all_questions, f, indent=2)

c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='A list of 10 questions about a specific topic' min_length=10 max_length=10 extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Generating questions for topic 19: Academic Reputation


c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='A list of 10 questions about a specific topic' min_length=10 max_length=10 extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Generating questions for topic 32: Specific Programs/Majors Offered


c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='A list of 10 questions about a specific topic' min_length=10 max_length=10 extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


...

Generating questions for topic 25: Campus Culture and Values


c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='A list of 10 questions about a specific topic' min_length=10 max_length=10 extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Generating questions for topic 26: Social Scene and Nightlife


c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='A list of 10 questions about a specific topic' min_length=10 max_length=10 extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\adity\Desktop\sutd_5055mlop\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Generating questions for topic 37: Accessibility and Disability Services


## Generate Answers

Now create answers for the questions. 

You can use the Google AI Studio Gemini model (assuming that they are good enough to generate good answers), your RAG system from assignment 3 or any other method you choose to generate answers for your question dataset.

Note: it is normal that some LLM calls fail, even with retry, so maybe you end up with less than 200 QA pairs but it should be at least 160 QA pairs.

In [ ]:
# QUESTION: Generate answers to al your questions using Gemini, your SUTD RAG system or any other method.
# Split your dataset in to 80% training and 20% test dataset.
# Store all questions and answer pairs in a huggingface dataset `sutd_qa_dataset` and push it to your Huggingface hub. 

#--- ADD YOUR SOLUTION HERE (40 points)---

# generate answers to al the questions using Gemini and then split the dataset in to 80% training and 20% test dataset.




In [ ]:
# test the chain
question = "When was SUTD founded?"

# Now run the answer generation chain
response = generate_answer(question)
print("\nModel Response:")
print(response["answer"])

In [ ]:
# now run the chain for all questions to collect context and generate answers



# Finetune Llama 3.2 1B model

Now use your SUTD QA dataset training data set to finetune a smaller Llama 3.2 1B LLM using parameter-efficient finetuning (PEFT). 
We recommend the unsloth library but you are free to choose other frameworks. You can decide the parameters for the finetuning. 
Push your finetuned model to Huggingface. 

Then we will compare the finetuned and non-finetuned LLMs with and without RAG to see if we were able to improve the SUTD chatbot answer quality. 


In [ ]:
# QUESTION: Finetune a Llama 3.2 1B model on the training split of your SUTD QA dataset.
# You need to prepare your dataset accordingly and set the hyperparameters for the training.
# Push your finetuned model to the Hugginface model hub {YOUR_HF_NAME}/llama-3.2-1B-sutdqa

#--- ADD YOUR SOLUTION HERE (50 points)---



In [ ]:
# QUESTION: Load a non-finetuned Llama 3.2 1B model and your finetuned SUTD QA Llama 3.2 1B model
# Ask it a simple test question (e.g. "What is special about SUTD?") to check that both models can generated answers

#--- ADD YOUR SOLUTION HERE (10 points)---



In [ ]:
# try out the llms

query = "What is special about SUTD?"

print("Question:", query)
response_base = llm_base.invoke(query,  pipeline_kwargs={"max_new_tokens": 512})
print("Answer base:", response_base)

print("---------")
response_finetune = llm_finetune.invoke(query, pipeline_kwargs={"max_new_tokens": 512})
print("Answer finetune:", response_finetune)


# Integrate and evaluate

Now integrate both the non-finetuned Llama 3.2 1B model and your finetuned model into your SUTD chatbot RAG system. 
Generate responses to the 20 questions you have collected in assignment 3 using these 4 appraoches
1. non-finetuned Llama 3.2 1B model without RAG
2. finetuned Llama 3.2 1B SUTD QA model without RAG
3. non-finetuned Llama 3.2 1B model with RAG
4. finetuned Llama 3.2 1B SUTD QA model with RAG

Compare the responses and decide what system produces the most accurate and high quality responses

In [ ]:
# QUESTION: Re-create the RAG chatbot system you have created in assignment 3 but with the Llama 3.2 1B (non-tuned and finetuned) models

#--- ADD YOUR SOLUTION HERE (40 points)---


# Bonus points: LLM-as-judge evaluation 

Implement an LLM-as-judge pipeline to assess the quality of the different system (finetuned vs. non-fintuned, RAG vs no RAG)



In [ ]:
# QUESTION: Implement an LLM-as-judge pipeline to assess the quality of the different system (finetuned vs. non-fintuned, RAG vs no RAG)

#--- ADD YOUR SOLUTION HERE (40 points)---

# Bonus points: chatbot UI

Implement a web UI frontend for your chatbot that you can demo in class. 


In [ ]:
# QUESTION: Implement a web UI frontend for your chatbot that you can demo in class. 

#--- ADD YOUR SOLUTION HERE (40 points)---

# End

This concludes assignment 4.

Please submit this notebook with your answers and the generated output cells as a **Jupyter notebook file** via github.


Every group member should do the following submission steps:
1. Create a private github repository **sutd_5055mlop** under your github user.
2. Add your instructors as collaborator: ddahlmeier and lucainiaoge
3. Save your submission as assignment_04_GROUP_NAME.ipynb where GROUP_NAME is the name of the group you have registered. 
4. Push the submission files to your repo 
5. Submit the link to the repo via eDimensions



**Assignment due 21 April 2025 11:59pm**